In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
# scale = 1/5 
# scale = 1/10
# scale = 1/20 
# scale = 1/40

In [3]:
# def tf_aug(X_in, scaler=scale):
#     X_out = np.zeros_like(X_in)
#     for channel in range(X_in.shape[-1]):
#         this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
#         scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
#         this_X = jitter(this_X, sigma=scale)
#         this_X = scaling(this_X, sigma=scale)
#         this_X = time_warp(this_X, sigma=scale)
#         this_X = window_warp(this_X, window_ratio=scale)
#         X_out[..., channel] = this_X[0, ..., 0]
#     return X_out


# def augment_this(X_in, y_in):
#     X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
#     return X_out, y_in


In [4]:
X, y, site = load_data()

2022-06-27 22:25:21.562101: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from sklearn.utils import shuffle, resample

In [6]:
from sklearn.metrics import r2_score, median_absolute_error

In [7]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [8]:
n_runs = 10

In [9]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [10]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    return pd.DataFrame(result)

In [11]:
dfs = []

In [12]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[1])
        dfs.append(this_eval)
        one_df = pd.concat(dfs)
        one_df.to_csv("lenet_2.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

10/10 [==============================] - 0s 20ms/step - loss: 16.3629 - mean_squared_error: 16.3629 - rmse: 4.0451 - mean_absolute_error: 3.1618 - val_loss: 12.7583 - val_mean_squared_error: 12.7583 - val_rmse: 3.5719 - val_mean_absolute_error: 2.7605 - lr: 0.0010
Epoch 21/1000
 8/10 [=======================>......] - ETA: 0s - loss: 15.7792 - mean_squared_error: 15.7792 - rmse: 3.9723 - mean_absolute_error: 3.1349
Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
10/10 [==============================] - 0s 19ms/step - loss: 15.4533 - mean_squared_error: 15.4533 - rmse: 3.9311 - mean_absolute_error: 3.1003 - val_loss: 12.8137 - val_mean_squared_error: 12.8137 - val_rmse: 3.5796 - val_mean_absolute_error: 2.7551 - lr: 0.0010
Epoch 22/1000
10/10 [==============================] - 0s 17ms/step - loss: 16.1260 - mean_squared_error: 16.1260 - rmse: 4.0157 - mean_absolute_error: 3.1402 - val_loss: 12.6643 - val_mean_squared_error: 12.6643 - val_rmse: 3.5587 - va

10/10 [==============================] - 0s 19ms/step - loss: 14.5268 - mean_squared_error: 14.5268 - rmse: 3.8114 - mean_absolute_error: 2.9753 - val_loss: 12.0818 - val_mean_squared_error: 12.0818 - val_rmse: 3.4759 - val_mean_absolute_error: 2.7175 - lr: 5.0000e-04
Epoch 49/1000
10/10 [==============================] - 0s 17ms/step - loss: 13.8728 - mean_squared_error: 13.8728 - rmse: 3.7246 - mean_absolute_error: 2.9257 - val_loss: 12.2683 - val_mean_squared_error: 12.2683 - val_rmse: 3.5026 - val_mean_absolute_error: 2.7031 - lr: 5.0000e-04
Epoch 50/1000
10/10 [==============================] - 0s 18ms/step - loss: 14.4517 - mean_squared_error: 14.4517 - rmse: 3.8015 - mean_absolute_error: 2.9986 - val_loss: 12.2824 - val_mean_squared_error: 12.2824 - val_rmse: 3.5046 - val_mean_absolute_error: 2.7012 - lr: 5.0000e-04
Epoch 51/1000
10/10 [==============================] - 0s 18ms/step - loss: 13.7510 - mean_squared_error: 13.7510 - rmse: 3.7082 - mean_absolute_error: 2.9129 - val_

Epoch 77/1000
10/10 [==============================] - 0s 19ms/step - loss: 13.1679 - mean_squared_error: 13.1679 - rmse: 3.6288 - mean_absolute_error: 2.7921 - val_loss: 10.6612 - val_mean_squared_error: 10.6612 - val_rmse: 3.2651 - val_mean_absolute_error: 2.5995 - lr: 5.0000e-04
Epoch 78/1000
10/10 [==============================] - 0s 18ms/step - loss: 12.6827 - mean_squared_error: 12.6827 - rmse: 3.5613 - mean_absolute_error: 2.7426 - val_loss: 10.5544 - val_mean_squared_error: 10.5544 - val_rmse: 3.2488 - val_mean_absolute_error: 2.5380 - lr: 5.0000e-04
Epoch 79/1000
10/10 [==============================] - 0s 18ms/step - loss: 13.6448 - mean_squared_error: 13.6448 - rmse: 3.6939 - mean_absolute_error: 2.8568 - val_loss: 10.5085 - val_mean_squared_error: 10.5085 - val_rmse: 3.2417 - val_mean_absolute_error: 2.5299 - lr: 5.0000e-04
Epoch 80/1000
10/10 [==============================] - 0s 17ms/step - loss: 11.8986 - mean_squared_error: 11.8986 - rmse: 3.4494 - mean_absolute_error:

Epoch 106/1000
10/10 [==============================] - 0s 18ms/step - loss: 10.8670 - mean_squared_error: 10.8670 - rmse: 3.2965 - mean_absolute_error: 2.5855 - val_loss: 9.5891 - val_mean_squared_error: 9.5891 - val_rmse: 3.0966 - val_mean_absolute_error: 2.3983 - lr: 5.0000e-04
Epoch 107/1000
10/10 [==============================] - 0s 17ms/step - loss: 10.6688 - mean_squared_error: 10.6688 - rmse: 3.2663 - mean_absolute_error: 2.5857 - val_loss: 9.6840 - val_mean_squared_error: 9.6840 - val_rmse: 3.1119 - val_mean_absolute_error: 2.4119 - lr: 5.0000e-04
Epoch 108/1000
10/10 [==============================] - 0s 18ms/step - loss: 10.9257 - mean_squared_error: 10.9257 - rmse: 3.3054 - mean_absolute_error: 2.5489 - val_loss: 9.5771 - val_mean_squared_error: 9.5771 - val_rmse: 3.0947 - val_mean_absolute_error: 2.4382 - lr: 5.0000e-04
Epoch 109/1000
10/10 [==============================] - 0s 18ms/step - loss: 11.7460 - mean_squared_error: 11.7460 - rmse: 3.4272 - mean_absolute_error: 2

...

10/10 [==============================] - 0s 17ms/step - loss: 7.0508 - mean_squared_error: 7.0508 - rmse: 2.6553 - mean_absolute_error: 2.0343 - val_loss: 7.0810 - val_mean_squared_error: 7.0810 - val_rmse: 2.6610 - val_mean_absolute_error: 1.9844 - lr: 5.0000e-04
Epoch 207/1000
10/10 [==============================] - 1s 74ms/step - loss: 7.0433 - mean_squared_error: 7.0433 - rmse: 2.6539 - mean_absolute_error: 2.0129 - val_loss: 6.6994 - val_mean_squared_error: 6.6994 - val_rmse: 2.5883 - val_mean_absolute_error: 1.9552 - lr: 5.0000e-04
Epoch 208/1000
10/10 [==============================] - 0s 22ms/step - loss: 7.2067 - mean_squared_error: 7.2067 - rmse: 2.6845 - mean_absolute_error: 2.0396 - val_loss: 6.8320 - val_mean_squared_error: 6.8320 - val_rmse: 2.6138 - val_mean_absolute_error: 1.9477 - lr: 5.0000e-04
Epoch 209/1000
10/10 [==============================] - 0s 18ms/step - loss: 7.0353 - mean_squared_error: 7.0353 - rmse: 2.6524 - mean_absolute_error: 2.0396 - val_loss: 6.821

10/10 [==============================] - 0s 19ms/step - loss: 7.2636 - mean_squared_error: 7.2636 - rmse: 2.6951 - mean_absolute_error: 2.0432 - val_loss: 6.4554 - val_mean_squared_error: 6.4554 - val_rmse: 2.5407 - val_mean_absolute_error: 1.9011 - lr: 5.0000e-04
Epoch 236/1000
10/10 [==============================] - 0s 18ms/step - loss: 6.6445 - mean_squared_error: 6.6445 - rmse: 2.5777 - mean_absolute_error: 1.9990 - val_loss: 6.6003 - val_mean_squared_error: 6.6003 - val_rmse: 2.5691 - val_mean_absolute_error: 1.9085 - lr: 5.0000e-04
Epoch 237/1000
10/10 [==============================] - 0s 19ms/step - loss: 7.0558 - mean_squared_error: 7.0558 - rmse: 2.6563 - mean_absolute_error: 2.0331 - val_loss: 6.7384 - val_mean_squared_error: 6.7384 - val_rmse: 2.5958 - val_mean_absolute_error: 1.9264 - lr: 5.0000e-04
Epoch 238/1000
10/10 [==============================] - 0s 18ms/step - loss: 6.5108 - mean_squared_error: 6.5108 - rmse: 2.5516 - mean_absolute_error: 1.9642 - val_loss: 6.876

10/10 [==============================] - 0s 18ms/step - loss: 6.3304 - mean_squared_error: 6.3304 - rmse: 2.5160 - mean_absolute_error: 1.9260 - val_loss: 6.6123 - val_mean_squared_error: 6.6123 - val_rmse: 2.5714 - val_mean_absolute_error: 1.8954 - lr: 2.5000e-04
Epoch 264/1000
10/10 [==============================] - 0s 17ms/step - loss: 6.0434 - mean_squared_error: 6.0434 - rmse: 2.4583 - mean_absolute_error: 1.8804 - val_loss: 6.5078 - val_mean_squared_error: 6.5078 - val_rmse: 2.5510 - val_mean_absolute_error: 1.8923 - lr: 2.5000e-04
Epoch 265/1000
10/10 [==============================] - 0s 17ms/step - loss: 5.8221 - mean_squared_error: 5.8221 - rmse: 2.4129 - mean_absolute_error: 1.8430 - val_loss: 6.5989 - val_mean_squared_error: 6.5989 - val_rmse: 2.5688 - val_mean_absolute_error: 1.8991 - lr: 2.5000e-04
Epoch 266/1000
 9/10 [==========================>...] - ETA: 0s - loss: 6.2759 - mean_squared_error: 6.2759 - rmse: 2.5052 - mean_absolute_error: 1.9329
Epoch 00266: ReduceLRO

Epoch 291/1000
10/10 [==============================] - 0s 18ms/step - loss: 6.4646 - mean_squared_error: 6.4646 - rmse: 2.5426 - mean_absolute_error: 1.9491 - val_loss: 6.4627 - val_mean_squared_error: 6.4627 - val_rmse: 2.5422 - val_mean_absolute_error: 1.8830 - lr: 6.2500e-05
Epoch 292/1000
10/10 [==============================] - 0s 17ms/step - loss: 6.4084 - mean_squared_error: 6.4084 - rmse: 2.5315 - mean_absolute_error: 1.9358 - val_loss: 6.4610 - val_mean_squared_error: 6.4610 - val_rmse: 2.5419 - val_mean_absolute_error: 1.8845 - lr: 6.2500e-05
Epoch 293/1000
10/10 [==============================] - 0s 18ms/step - loss: 6.2368 - mean_squared_error: 6.2368 - rmse: 2.4974 - mean_absolute_error: 1.9225 - val_loss: 6.4487 - val_mean_squared_error: 6.4487 - val_rmse: 2.5394 - val_mean_absolute_error: 1.8843 - lr: 6.2500e-05
Epoch 294/1000
10/10 [==============================] - 0s 17ms/step - loss: 5.7802 - mean_squared_error: 5.7802 - rmse: 2.4042 - mean_absolute_error: 1.8776 - 

10/10 [==============================] - 0s 21ms/step - loss: 6.0296 - mean_squared_error: 6.0296 - rmse: 2.4555 - mean_absolute_error: 1.8744 - val_loss: 6.5582 - val_mean_squared_error: 6.5582 - val_rmse: 2.5609 - val_mean_absolute_error: 1.8872 - lr: 3.1250e-05
Epoch 320/1000
10/10 [==============================] - 0s 17ms/step - loss: 5.4638 - mean_squared_error: 5.4638 - rmse: 2.3375 - mean_absolute_error: 1.7897 - val_loss: 6.5624 - val_mean_squared_error: 6.5624 - val_rmse: 2.5617 - val_mean_absolute_error: 1.8858 - lr: 3.1250e-05
Epoch 321/1000
10/10 [==============================] - 0s 18ms/step - loss: 5.9520 - mean_squared_error: 5.9520 - rmse: 2.4397 - mean_absolute_error: 1.8854 - val_loss: 6.4864 - val_mean_squared_error: 6.4864 - val_rmse: 2.5468 - val_mean_absolute_error: 1.8812 - lr: 3.1250e-05
Epoch 322/1000
10/10 [==============================] - 0s 19ms/step - loss: 6.1573 - mean_squared_error: 6.1573 - rmse: 2.4814 - mean_absolute_error: 1.8846 - val_loss: 6.452